### 1. Binary Cross Entropy Loss (BCE)

- ใช้สำหรับงาน Binary Classification หรือการจำแนกที่มีเพียง 2 ตัวเลือก (เช่น เป็นสแปม หรือ ไม่เป็นสแปม) 
Activation Function: มักใช้คู่กับ Sigmoid ในเลเยอร์สุดท้าย ซึ่งจะให้ค่าความน่าจะเป็นระหว่าง 0 ถึง 1 สำหรับคลาสบวก (Positive Class)

![alt text](<สกรีนช็อต 2025-12-26 015526.png>)

- โค้ดนี้คือการสร้างโครงข่ายประสาทเทียม (Neural Network) ด้วย PyTorch เพื่อทำนายการย้ายค่ายของลูกค้า (Customer Churn) ซึ่งเป็นปัญหาแบบ Binary Classification (ย้าย หรือ ไม่ย้าย) 

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('churn_data.csv')
df = pd.get_dummies(df, columns=['ContractType'], drop_first=True)
X = df.drop('Churn', axis=1).values
y = df['Churn'].values
scaler = StandardScaler()
X = scaler.fit_transform(X)

![alt text](<สกรีนช็อต 2025-12-26 154704.png>)

In [3]:
import torch
from torch.utils.data import TensorDataset, DataLoader

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)

![alt text](<สกรีนช็อต 2025-12-26 154718.png>)

In [4]:
dataset = TensorDataset(X_train, y_train)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

import torch.nn as nn

class ChurnNet(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )
    
    def forward(self, x):
        return self.layers(x)

model = ChurnNet(input_dim=X_train.shape[1])


![alt text](<สกรีนช็อต 2025-12-26 154727.png>)

In [5]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = torch.sigmoid(model(inputs))
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 1, Loss: 0.8368
Epoch 2, Loss: 0.8328
Epoch 3, Loss: 0.8289
Epoch 4, Loss: 0.8250
Epoch 5, Loss: 0.8211
Epoch 6, Loss: 0.8173
Epoch 7, Loss: 0.8135
Epoch 8, Loss: 0.8098
Epoch 9, Loss: 0.8061
Epoch 10, Loss: 0.8025


![alt text](<สกรีนช็อต 2025-12-26 154735.png>)
![alt text](<สกรีนช็อต 2025-12-26 154744.png>)
![alt text](<สกรีนช็อต 2025-12-26 154752.png>)

### 2. Multiclass Classification Example on Iris Dataset

#### Data Preparation

In [ ]:
from sklearn.datasets import load_iris

# โหลด Dataset มาตรฐานที่มีข้อมูลดอกไม้ 150 ตัวอย่าง
iris = load_iris()

X = iris.data
y = iris.target

# ปรับสเกลข้อมูล (Standardization) ให้มีค่าเฉลี่ยเป็น 0 และส่วนเบี่ยงเบนมาตรฐานเป็น 1 
# เพื่อให้ Neural Network ประมวลผลได้เสถียรขึ้น Scikit-learn Scaler
scaler = StandardScaler()

#  แบ่งข้อมูลเป็นชุดสอน (Train) 80% และชุดทดสอบ (Test) 20% โดยใช้ random_state=42 
# เพื่อให้ผลการสุ่มคงที่ทุกครั้งที่รัน
X = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# dtype=torch.float32: 
# ข้อมูล Input ต้องเป็นเลขทศนิยมความละเอียดสูงเพื่อใช้ในการคำนวณ Weight
X_train = torch.tensor(X_train, dtype=torch.float32)

# ข้อมูล Label ในงาน Multi-class ต้องเป็นเลขจำนวนเต็ม (Integer) 
# เพื่อใช้เป็นดัชนีอ้างอิงคลาสใน Loss Function
y_train = torch.tensor(y_train, dtype=torch.long)

dataset = TensorDataset(X_train, y_train)

# ทำหน้าที่แบ่งข้อมูลเข้าโมเดลทีละ 16 ตัวอย่าง (Batch Size) และสลับตำแหน่งข้อมูล (shuffle=True) 
# เพื่อป้องกันโมเดลจำลำดับข้อมูล PyTorch DataLoader Docs
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
class IrisNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 10),   # รับ 4 คุณสมบัติขยายไป 10 Node
            nn.ReLU(),                  # ตัดค่าที่ติดลบให้กลายเป็น 0 เพื่อสร้าง Non Linear
            nn.Linear(10, output_dim)   # ลดลงเหลือ 3 Node (ตามจำนวนสายพันธุ์ดอกไม้)
        )

    # ฟังก์ชันที่กำหนดทิศทางไหลของข้อมูลจาก Input ผ่าน Layer ต่างๆ ไปจนถึง Output
    def forward(self, x):
        return self.layers(x)
    
model = IrisNet(input_dim=4, output_dim=3)

#  ฟังก์ชันวัดความผิดพลาดที่รวม Softmax ไว้ในตัวแล้ว 
# เหมาะสำหรับงานจำแนกประเภทหลายคลาส (Multi-class)
criterion = nn.CrossEntropyLoss()

# อัลกอริทึมสำหรับปรับน้ำหนักของโมเดลให้แม่นยำขึ้น โดยใช้ Learning Rate (lr) เท่ากับ 0.01
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
for epoch in range(15):                                # ทำซ้ำข้อมูลทั้งหมด 15 รอบ
    for inputs, targets in dataloader:                 
        optimizer.zero_grad()                          # ล้างค่าความชันเก่า
        outputs = model(inputs)                        # ทำนายผล (Forward Pass)
        loss = criterion(outputs, targets)             # คำนวณความผิดพลาด
        loss.backward()                                # คำนวณหาทิศทางที่จะปรับน้ำหนัก
        optimizer.step()                               # ปรับน้ำหนักทิศทาง Model

    # loss.item(): ดึงค่าความผิดพลาดออกมาแสดงผลในแต่ละรอบ 
    # เพื่อให้ผู้พัฒนาทราบว่าโมเดลกำลังเก่งขึ้นหรือไม่ (ค่า Loss ต้องลดลง)
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}") 

Epoch 1, Loss: 0.8388
Epoch 2, Loss: 0.7887
Epoch 3, Loss: 0.5463
Epoch 4, Loss: 0.3030
Epoch 5, Loss: 0.5417
Epoch 6, Loss: 0.4801
Epoch 7, Loss: 0.5920
Epoch 8, Loss: 0.1863
Epoch 9, Loss: 0.3020
Epoch 10, Loss: 0.1410
Epoch 11, Loss: 0.2914
Epoch 12, Loss: 0.0955
Epoch 13, Loss: 0.3224
Epoch 14, Loss: 0.0580
Epoch 15, Loss: 0.2456
